In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, warnings, scipy 
#import plotly.offline as py
#import plotly.figure_factory as ff
#py.init_notebook_mode(connected=True)
#import plotly.graph_objs as go
#from plotly import tools
#from plotly.subplots import make_subplots
from pyspark.sql import SparkSession
from pyspark.sql import Row
import numpy as np
import pandas as pd
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
import pyspark.sql.functions as fn
from pyspark.sql.functions import exp, corr
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import isnan, when, count, col, isnull

In [2]:
# Do not delete or change this cell

import os

# Define a function to determine if we are running on data bricks
# Return true if running in the data bricks environment, false otherwise
def is_databricks():
    # get the databricks runtime version
    db_env = os.getenv("DATABRICKS_RUNTIME_VERSION")
    
    # if running on data bricks
    if db_env != None:
        return True
    else:
        return False

# Define a function to read the data file.  The full path data file name is constructed
# by checking runtime environment variables to determine if the runtime environment is 
# databricks, or a student's personal computer.  The full path file name is then
# constructed based on the runtime env.
# 
# Params
#   data_file_name: The base name of the data file to load
# 
# Returns the full path file name based on the runtime env
#
def get_training_filename(data_file_name):    
    # if running on data bricks
    if is_databricks():
        # build the full path file name assuming data brick env
        full_path_name = "/FileStore/tables/%s" % data_file_name
    # else the data is assumed to be in the same dir as this notebook
    else:
        # Assume the student is running on their own computer and load the data
        # file from the same dir as this notebook
        full_path_name = data_file_name
    
    # return the full path file name to the caller
    return full_path_name

In [3]:
#DirectoryPath="D:\\BDAPROJECT\\data\\"  #Lakshya Directory path
#DirectoryPath=""  #Sankalp Directory path
DirectoryPath="/FileStore/tables"  #Abhiraj Directory path
#DirectoryPath=""  #Surjit Directory path

In [4]:
flightPath=DirectoryPath+"flights.csv"
airportPath=DirectoryPath+"airports.csv"
airlinesPath=DirectoryPath+"airlines.csv"

In [5]:
airportPath = spark.read.option('delimiter' , ',').csv(get_training_filename('airports.csv'),inferSchema = True, header=True)
flightPath = spark.read.option('delimiter' , ',').csv(get_training_filename('flights.csv'),inferSchema = True, header=True)
airlinesPath = spark.read.option('delimiter' , ',').csv(get_training_filename('airlines.csv'),inferSchema = True, header=True)


In [6]:
print(flightPath.count(),len(flightPath.columns))

5819079 31

In [7]:
flightPath.columns

Out[7]: ['YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'AIRLINE',
 'FLIGHT_NUMBER',
 'TAIL_NUMBER',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_TIME',
 'DEPARTURE_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'WHEELS_ON',
 'TAXI_IN',
 'SCHEDULED_ARRIVAL',
 'ARRIVAL_TIME',
 'ARRIVAL_DELAY',
 'DIVERTED',
 'CANCELLED',
 'CANCELLATION_REASON',
 'AIR_SYSTEM_DELAY',
 'SECURITY_DELAY',
 'AIRLINE_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'WEATHER_DELAY']

In [8]:
#CHECK FOR THE NULL VALUES IN THE COLUMNS
flightPath.select([count(when(isnull(c), c)).alias(c) for c in flightPath.columns]).show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
 0| 0| 0| 0| 0| 0| 14721| 0| 0| 0| 86153| 86153| 89047| 89047| 6| 105071| 105071| 0| 92513| 92513| 0| 92513| 105071| 0| 0| 5729195| 4755640| 4755640| 4755640| 4755640| 4755640|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+

In [9]:
flightPath.printSchema

Out[9]: <bound method DataFrame.printSchema of DataFrame[YEAR: int, MONTH: int, DAY: int, DAY_OF_WEEK: int, AIRLINE: string, FLIGHT_NUMBER: int, TAIL_NUMBER: string, ORIGIN_AIRPORT: string, DESTINATION_AIRPORT: string, SCHEDULED_DEPARTURE: int, DEPARTURE_TIME: int, DEPARTURE_DELAY: int, TAXI_OUT: int, WHEELS_OFF: int, SCHEDULED_TIME: int, ELAPSED_TIME: int, AIR_TIME: int, DISTANCE: int, WHEELS_ON: int, TAXI_IN: int, SCHEDULED_ARRIVAL: int, ARRIVAL_TIME: int, ARRIVAL_DELAY: int, DIVERTED: int, CANCELLED: int, CANCELLATION_REASON: string, AIR_SYSTEM_DELAY: int, SECURITY_DELAY: int, AIRLINE_DELAY: int, LATE_AIRCRAFT_DELAY: int, WEATHER_DELAY: int]>

In [10]:
#SHOWS COUNT OF DISTINCT AIRLINES WITH THEIR ABBERIVIATIONS
flightPath.select('AIRLINE').distinct().show()
flightPath.select('AIRLINE').distinct().count()

+-------+
AIRLINE|
+-------+
 UA|
 NK|
 AA|
 EV|
 B6|
 DL|
 OO|
 F9|
 US|
 MQ|
 HA|
 AS|
 VX|
 WN|
+-------+

Out[10]: 14

In [11]:
#SHOWS THE TOTAL/MOST CANCELATIONS BY INDIVIDUAL AIRLINES

df = flightPath.groupBy('AIRLINE').sum('CANCELLED').toPandas()
df
ax = sns.barplot(x = 'AIRLINE', y = 'sum(CANCELLED)', data = df)
display()

perc = flightPath.groupBy('AIRLINE')

In [12]:
#SHOWS THE AIRLINE WITH MAXIMUM/MINIMUM DELAYS

df1 = flightPath.groupBy('AIRLINE').sum('AIRLINE_DELAY').toPandas()
df1
fig2 = sns.barplot(x = 'AIRLINE', y = 'sum(AIRLINE_DELAY)', data = df1)
display()

In [13]:
#SHOWS WHICH DAY HAS MAXIMUM/MINIMUM DELAY

df2 = flightPath.groupBy('DAY_OF_WEEK').avg('AIRLINE_DELAY').toPandas()
df2
fig3 = sns.barplot(x = 'DAY_OF_WEEK', y = 'avg(AIRLINE_DELAY)', data = df2)
display()

In [14]:
#SHOWS WHICH MONTH HAS MAXIMUM/MINIMUM DELAY

df3 = flightPath.groupBy('MONTH').avg('AIRLINE_DELAY').toPandas()
df3
fig4 = sns.barplot(x = 'MONTH', y = 'avg(AIRLINE_DELAY)', data = df3)
display()


In [15]:
df5 =  flightPath.groupBy('AIRLINE').avg('LATE_AIRCRAFT_DELAY')
df5 = df5.toPandas()
fig4 = sns.barplot(x = 'AIRLINE', y = 'avg(LATE_AIRCRAFT_DELAY)', data = df5)
display()

In [16]:
#CHECK FOR THE NULL VALUES IN THE COLUMNS of FLIGHT_CANCEL
flights_cancel.select([count(when(isnull(c), c)).alias(c) for c in flights_cancel.columns]).show()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-597535771010325> in <module> 
 1 #CHECK FOR THE NULL VALUES IN THE COLUMNS of FLIGHT_CANCEL 
 ----> 2 flights_cancel . select ( [ count ( when ( isnull ( c ) , c ) ) . alias ( c ) for c in flights_cancel . columns ] ) . show ( ) 

 NameError : name 'flights_cancel' is not defined

In [17]:
#drop the columns with more than 90% NA values
# lst = ['CANCELLATION_REASON','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']

# for i in lst:
#   flightPath= flightPath.drop(i)
# flightPath.columns  

# #drop other columns which we might not take under consideration

# lst2 = ['TAXI_OUT','WHEELS_OFF','TAXI_IN','WHEELS_ON']
# for j in lst2:
#   flightPath= flightPath.drop(j)
# flightPath.columns  

In [18]:
df_1 = flightPath.groupBy('AIRLINE').count()
df_1.show()

+-------+-------+
AIRLINE| count|
+-------+-------+
 UA| 515723|
 NK| 117379|
 AA| 725984|
 EV| 571977|
 B6| 267048|
 DL| 875881|
 OO| 588353|
 F9| 90836|
 US| 198715|
 MQ| 294632|
 HA| 76272|
 AS| 172521|
 VX| 61903|
 WN|1261855|
+-------+-------+

In [19]:
flightPath.select('SCHEDULED_DEPARTURE').show(5)
flightPath.select('SCHEDULED_ARRIVAL').show(5)
flightPath.select('DEPARTURE_TIME').show(5)
flightPath.select('ARRIVAL_TIME').show(5)


In [20]:
flightPath.select('SCHEDULED_DEPARTURE').show()

In [21]:
categ = flightPath.select('AIRLINE').distinct().rdd.flatMap(lambda x:x).collect()
exprs = [fn.when(fn.col('AIRLINE') == cat,1).otherwise(0)\
            .alias(str(cat)) for cat in categ]
df = flightPath.select(exprs+flightPath.columns)

In [22]:
df.select('UA','NK','AA').show()

In [23]:
flights_df = flightPath.select('MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'ARRIVAL_DELAY', 'CANCELLED')

flights_df = flights_df.filter((fn.col('CANCELLED')==0))

flights_df = flights_df.withColumn("Flight_Delayed", fn.when(fn.col("DEPARTURE_DELAY")<10, 0).otherwise(1))

In [24]:
print(flights_df.count(), len(flights_df.columns))

5729195 13

In [25]:
from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splits=[ 0, 100, 1000, float('Inf') ],inputCol="DISTANCE", outputCol="Distance_Bucket")
flights_df = bucketizer.setHandleInvalid("keep").transform(flights_df)

In [26]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

t = {0.0:"Short", 1.0: "Medium", 2.0:"Long"}
udf_foo = udf(lambda x: t[x], StringType())
flights_df = flights_df.withColumn("Flight_Distance", udf_foo("Distance_Bucket"))
flights_df.show()

+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+-------------+---------+--------------+---------------+---------------+
MONTH|DAY|DAY_OF_WEEK|AIRLINE|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_DELAY|DISTANCE|SCHEDULED_ARRIVAL|ARRIVAL_DELAY|CANCELLED|Flight_Delayed|Distance_Bucket|Flight_Distance|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+-------------+---------+--------------+---------------+---------------+
 1| 1| 4| AS| ANC| SEA| 5| -11| 1448| 430| -22| 0| 0| 2.0| Long|
 1| 1| 4| AA| LAX| PBI| 10| -8| 2330| 750| -9| 0| 0| 2.0| Long|
 1| 1| 4| US| SFO| CLT| 20| -2| 2296| 806| 5| 0| 0| 2.0| Long|
 1| 1| 4| AA| LAX| MIA| 20| -5| 2342| 805| -9| 0| 0| 2.0| Long|
 1| 1| 4| AS| SEA| ANC| 25| -1| 1448| 320| -21| 0| 0| 2.0| Long|
 1| 1| 4| DL| SFO| MSP| 25| -5| 1589| 602| 8| 0| 0| 2.0| Long|
 1| 1| 4| NK| LAS| MSP| 25| -6| 1299| 526| -17| 0| 0| 2.0| Long|
 1| 1| 4| US| LAX| CLT| 30| 14| 2125| 803| -10| 0| 1| 2.0| Long|
 1| 1| 4| AA| SFO| DFW| 30| -11| 1464| 545| -13| 0| 0| 2.0| Long|
 1| 1| 4| DL| LAS| ATL| 30| 3| 1747| 711| -15| 0| 0| 2.0| Long|
 1| 1| 4| DL| DEN| ATL| 30| -6| 1199| 523| -30| 0| 0| 2.0| Long|
 1| 1| 4| AA| LAS| MIA| 35| -8| 2174| 803| -10| 0| 0| 2.0| Long|
 1| 1| 4| DL| LAX| MSP| 35| 0| 1535| 609| -4| 0| 0| 2.0| Long|
 1| 1| 4| DL| SLC| ATL| 40| -6| 1590| 615| -22| 0| 0| 2.0| Long|
 1| 1| 4| DL| SEA| MSP| 40| -1| 1399| 549| 8| 0| 0| 2.0| Long|
 1| 1| 4| AS| ANC| SEA| 45| -4| 1448| 509| -14| 0| 0| 2.0| Long|
 1| 1| 4| DL| ANC| SEA| 45| -14| 1448| 515| -24| 0| 0| 2.0| Long|
 1| 1| 4| UA| SFO| IAH| 48| -6| 1635| 626| -7| 0| 0| 2.0| Long|
 1| 1| 4| AS| ANC| PDX| 50| -4| 1542| 525| -18| 0| 0| 2.0| Long|
 1| 1| 4| DL| PDX| MSP| 50| -5| 1426| 603| -12| 0| 0| 2.0| Long|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+-------------+---------+--------------+---------------+---------------+
only showing top 20 rows

In [27]:
#StringIndexer encodes a string column of labels to a column of label indices.
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="AIRLINE", outputCol="Airline_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

In [28]:
#perform one-hot encoding for Airline Columns
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCol="Airline_Numeric", outputCol="Airline_OHE")
flights_df= encoder.transform(flights_df)

In [29]:
#StringIndexer encodes a string column of labels to a column of label indices.

indexer = StringIndexer(inputCol="ORIGIN_AIRPORT", outputCol="OA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

#perform one-hot encoding for Origin Airport Columns

encoder = OneHotEncoder(inputCol="OA_Numeric", outputCol="Origin_Airport_OHE")
flights_df= encoder.transform(flights_df)


In [30]:
indexer = StringIndexer(inputCol="DESTINATION_AIRPORT", outputCol="DA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

encoder = OneHotEncoder(inputCol="DA_Numeric", outputCol="Destination_Airport_OHE")
flights_df= encoder.transform(flights_df)

In [31]:
#Drop ARRIVAL_Delay as this columns will help to determine the delay which we were to determine

flights_df = flights_df.drop('ARRIVAL_DELAY')

In [32]:
flights_df.columns

Out[27]: ['MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'AIRLINE',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_DELAY',
 'DISTANCE',
 'SCHEDULED_ARRIVAL',
 'CANCELLED',
 'Flight_Delayed',
 'Distance_Bucket',
 'Flight_Distance',
 'Airline_Numeric',
 'Airline_OHE',
 'OA_Numeric',
 'Origin_Airport_OHE',
 'DA_Numeric',
 'Destination_Airport_OHE']

In [33]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(
    inputCols=["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL", "CANCELLED"], outputCol="features")

In [34]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StandardScaler

features = flights_df.columns
lr_data = flights_df.select(col("Flight_Delayed").alias("label"), *features)
lr_data.printSchema()


train, test = lr_data.randomSplit([0.95, 0.05], seed=12345)

vectorAssembler = VectorAssembler(inputCols=["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL", "CANCELLED"], outputCol="unscaled_features")

standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")

lr = LogisticRegression(featuresCol='features', labelCol='label')
lr_pipeline = Pipeline(stages=[vectorAssembler,standardScaler, lr])

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()
crossval = CrossValidator(estimator=lr_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)
#cvModel = crossval.fit(train)


root
-- label: integer (nullable = false)
-- MONTH: integer (nullable = true)
-- DAY: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- AIRLINE: string (nullable = true)
-- ORIGIN_AIRPORT: string (nullable = true)
-- DESTINATION_AIRPORT: string (nullable = true)
-- SCHEDULED_DEPARTURE: integer (nullable = true)
-- DEPARTURE_DELAY: integer (nullable = true)
-- DISTANCE: integer (nullable = true)
-- SCHEDULED_ARRIVAL: integer (nullable = true)
-- CANCELLED: integer (nullable = true)
-- Flight_Delayed: integer (nullable = false)
-- Distance_Bucket: double (nullable = true)
-- Flight_Distance: string (nullable = true)
-- Airline_Numeric: double (nullable = false)
-- Airline_OHE: vector (nullable = true)
-- OA_Numeric: double (nullable = false)
-- Origin_Airport_OHE: vector (nullable = true)
-- DA_Numeric: double (nullable = false)
-- Destination_Airport_OHE: vector (nullable = true)

In [35]:
cvModel = crossval.fit(train)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [36]:
prediction = cvModel.transform(test)
prediction.limit(5).toPandas()

,label,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,DISTANCE,SCHEDULED_ARRIVAL,CANCELLED,Flight_Delayed,Distance_Bucket,Flight_Distance,Airline_Numeric,Airline_OHE,OA_Numeric,Origin_Airport_OHE,DA_Numeric,Destination_Airport_OHE,unscaled_features,features,rawPrediction,probability,prediction
0,0,1,1,4,AA,ATL,DFW,1640,-5,731,1805,0,0,1.0,Medium,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 1.0, 4.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.2943759827984922, 0.11397202949278949, 2.01...","[0.9024066040167309, -0.9024066040167309]","[0.7114438093058966, 0.2885561906941034]",0.0
1,0,1,1,4,AA,ATL,MIA,815,-9,594,1012,0,0,1.0,Medium,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",24.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 1.0, 4.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.2943759827984922, 0.11397202949278949, 2.01...","[1.7441250172058136, -1.7441250172058136]","[0.8512102617133985, 0.14878973828660158]",0.0
2,0,1,1,4,AA,BWI,MIA,1309,-9,946,1550,0,0,1.0,Medium,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",19.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",24.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 1.0, 4.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.2943759827984922, 0.11397202949278949, 2.01...","[1.1105815817090996, -1.1105815817090996]","[0.7522375203480615, 0.2477624796519385]",0.0
3,0,1,1,4,AA,CLT,DFW,825,0,936,1030,0,0,1.0,Medium,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",14.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 1.0, 4.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.2943759827984922, 0.11397202949278949, 2.01...","[1.676453009441589, -1.676453009441589]","[0.8424342796371611, 0.15756572036283892]",0.0
4,0,1,1,4,AA,DCA,MIA,840,-1,919,1133,0,0,1.0,Medium,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",22.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",24.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 1.0, 4.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.2943759827984922, 0.11397202949278949, 2.01...","[1.9055708727766874, -1.9055708727766874]","[0.8705207402635151, 0.12947925973648486]",0.0


In [37]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
mce = MulticlassClassificationEvaluator(labelCol='label', predictionCol="prediction")

mce.evaluate(prediction, {mce.metricName: "f1"})


Out[50]: 0.6792220621362316

In [38]:
bce = BinaryClassificationEvaluator(labelCol='label')
bce.evaluate(prediction, {bce.metricName: "areaUnderROC"} )

Out[56]: 0.6633127774575106